# SemEval-2020 Task 4: Commonsense Validation and Explanation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1rlwPIErsvGcYYgM6s_FT9FDn9lcHzvVU?usp=sharing)

## Getting the data

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
import pandas as pd
import numpy as np

In [5]:
X_train = pd.read_csv('/content/gdrive/MyDrive/data/Training/subtaskA_data_all.csv', index_col=0)
X_dev   = pd.read_csv('/content/gdrive/MyDrive/data/Dev/subtaskA_dev_data.csv'     , index_col=0)
X_test  = pd.read_csv('/content/gdrive/MyDrive/data/Test/subtaskA_test_data.csv'   , index_col=0)

y_train = pd.read_csv('/content/gdrive/MyDrive/data/Training/subtaskA_answers_all.csv', index_col=0, header=None)
y_dev   = pd.read_csv('/content/gdrive/MyDrive/data/Dev/subtaskA_gold_answers.csv'    , index_col=0, header=None)
y_test  = pd.read_csv('/content/gdrive/MyDrive/data/Test/subtaskA_gold_answers.csv'   , index_col=0, header=None)

## Not a fan of pandas, so just switch to numpy arrays

In [6]:
X_train = np.array([X_train['sent0'], X_train['sent1']]).T
y_train = np.array(y_train)
X_test = np.array([X_test['sent0'], X_test['sent1']]).T
y_test = np.array(y_test)

In [7]:
# a quick glance at the data
print(X_train[0][0])
print(X_train[0][1])
print(y_train[0][0])

print()

print(X_test[0][0])
print(X_test[0][1])
print(y_test[0][0])

He poured orange juice on his cereal.
He poured milk on his cereal.
0

He loves to stroll at the park with his bed
He loves to stroll at the park with his dog.
0


# A pre-trained LM model, just out of curiosity

In [8]:
!pip install lm-scorer
from lm_scorer.models.auto import AutoLMScorer
import torch

     |████████████████████████████████| 1.5MB 5.2MB/s 
     |████████████████████████████████| 675kB 16.8MB/s 
     |████████████████████████████████| 1.2MB 23.0MB/s 
     |████████████████████████████████| 870kB 31.6MB/s 
     |████████████████████████████████| 5.6MB 45.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=46b428754c648ce888a55907610dcb82f2e4dbf7b8fbd7bb694f2b3f8216b7a9
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [9]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)
scorer = AutoLMScorer.from_pretrained("gpt2-large", device=device)

sent0 = "He put the elephant in the fridge"
sent1 = "SKrskr riječii smisla neimadeju"
sent2 = "He put the turkey in the fridge"

print(sent0, scorer.sentence_score(sent0))
print(sent1, scorer.sentence_score(sent1))
print(sent2, scorer.sentence_score(sent2))

cpu



He put the elephant in the fridge 1.297326704875864e-18
SKrskr riječii smisla neimadeju 0.0
He put the turkey in the fridge 8.462602778441146e-18


In [10]:
def score(X):
    return np.array([ [scorer.sentence_score(x[0]), scorer.sentence_score(x[1]) ] for x in X])

def predict(X):
    scores = score(X)
    # the label is the "incorrect" sentence, so minimum probability
    return np.argmin(scores, axis=1)


In [11]:
from sklearn.metrics import accuracy_score

y_test_pred = predict(X_test)

accuracy = accuracy_score(y_test, y_test_pred)
print(f"acc = {accuracy}")

acc = 0.788


# My ngram language model

In [12]:
# I had problems with the installed version
!pip install "nltk==3.4.4"
from nltk import download
download('brown')

from nltk.corpus import brown
from collections import defaultdict
from nltk.util import ngrams as util_ngrams

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [13]:
class MyCounter:

    def __init__(self, ngrams):
        # dictionary where the key is ngram length and value is a dictionary
        # where the key is the ngram and the value is occurrence count
        dictionary = defaultdict(lambda: defaultdict(lambda: 0))
        for ngram in ngrams:
            dictionary[len(ngram)][ngram] += 1
        
        self.dictionary = dictionary

    def count(self, ngram: tuple):
        """Count how many times ngram appears."""
        return self.dictionary[len(ngram)][ngram]

    def count_ngrams(self, n: int):
        """Count different ngrams of length n."""
        return len(self.dictionary[n])


class NgramLanguageModel:

    def __init__(self, n, sentences):
        m = n - 1
        mgram_generators = [util_ngrams(sentence, m) for sentence in sentences]
        ngram_generators = [util_ngrams(sentence, n) for sentence in sentences]
        mgrams = [mgram for mgram_gen in mgram_generators for mgram in mgram_gen]
        ngrams = [ngram for ngram_gen in ngram_generators for ngram in ngram_gen]
        
        self.counter = MyCounter(mgrams + ngrams)
        self.n = n

    def ngram_score(self, ngram):
        ngram_count = self.counter.count(ngram)
        mgram = tuple(ngram[:-1])
        mgram_count = self.counter.count(mgram)
        # add one smoothing
        return (ngram_count + 1) / (mgram_count + self.counter.count_ngrams(self.n))

    def log_sentence_score(self, sentence):
        probs = [self.ngram_score(ngram) for ngram in util_ngrams(sentence, self.n)]
        sentence_prob = np.sum(np.log(probs))
        return sentence_prob

    def sentence_score(self, sentence):
        return np.exp(log_sentence_score(sentence))


In [14]:
lm = NgramLanguageModel(3, brown.sents())
# sanity check
print("shit on desk:", lm.ngram_score(('shit', 'on', 'desk')))
print("at that time:", lm.ngram_score(('at', 'that', 'time')))

def log_score(X):
    log_probs = [[lm.log_sentence_score(x[0]), lm.log_sentence_score(x[1])] for x in X]
    return log_probs

def predict(X):
    log_probs = log_score(X)
    # the correct label is the "wrong" sentence, so min prob
    return np.argmin(log_probs, axis=1)

sent0 = "I put an elephant in the fridge"
sent1 = "I put a turkey in the fridge"
sent2 = "I shit on the desk"
print(sent0, lm.log_sentence_score(sent0))
print(sent1, lm.log_sentence_score(sent1))
print(sent2, lm.log_sentence_score(sent2))

shit on desk: 1.197151736588309e-06
at that time: 2.513889238040172e-05
I put an elephant in the fridge -395.43191664017684
I put a turkey in the fridge -354.52578555293314
I shit on the desk -218.1699055374355


In [15]:
# calculate accuracy on test
y_test_predicted = predict(X_test)
accuracy = accuracy_score(y_test, y_test_predicted)
print(accuracy)

0.53
